<a href="https://colab.research.google.com/github/mykolesiko/eeg_investigation/blob/diplom/MADE_DE_cross_subject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook, we will show how to read DE features with python

- Author: Wei Liu
- Affiliation: [BCMI lab, Shanghai Jiao Tong University, Shanghai, China](http://bcmi.sjtu.edu.cn)
- Date: May, 11, 2021

In [1]:
import numpy as np
import pickle

# check the version of these modules
print(np.__version__)
print(pickle.format_version)

1.19.5
4.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [3]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.epochs = 2
    self.batch_size = 8
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/seed_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [4]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/seed/EEG_DE_features")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# load DE features named '1_123.npz'
data_seed = []
labels_seed = []
for i in range(1, 16):
    file_name = f'{i}_123.npz' 
    print(f"loading {file_name}")
    data_npz = np.load(file_name)
    data_seed.append(pickle.loads(data_npz['data']))
    labels_seed.append(pickle.loads(data_npz['label']))
    #print(data_npz.files)

loading 1_123.npz
loading 2_123.npz
loading 3_123.npz
loading 4_123.npz
loading 5_123.npz
loading 6_123.npz
loading 7_123.npz
loading 8_123.npz
loading 9_123.npz
loading 10_123.npz
loading 11_123.npz
loading 12_123.npz
loading 13_123.npz
loading 14_123.npz
loading 15_123.npz


In [7]:
# As we can see, there are 45 keys in both 'data' and 'label'.
# Each participant took part in our experiments for 3 sessions, and he/she watched 15 movie clips (i.e. 15 trials) during each session.
# Therefore, we could extract 3 * 15 = 45 DE feature matrices.

# The key indexes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] belong to Session 1.
# The key indexes [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] belong to Session 2.
# The key indexes [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44] belong to Session 3.

# We will print the emotion labels for each trial.
# label_dict = {0:'Disgust', 1:'Fear', 2:'Sad', 3:'Neutral', 4:'Happy'}
# for i in range(45):
#     print('Session {} -- Trial {} -- EmotionLabel : {}'.format(i//15+1, i%15+1, label_dict[label[i][0]]))

In [8]:
# def get_label_valence(l):
#   if (l == 0) or (l == 1) or (l == 2):
#     return 0
#   elif (l == 3):
#     return 1
#   else:
#     return 2 

# def get_label_valence(l):
#    if (l == 0) or (l == 1):
#       return 0
#    elif (l == 2):
#       return 1
#    elif (l == 3):
#       return 2
#    else:
#       return 3  

def get_label(l):
  if (l == 0) or (l == 1) or (l == 2):
     return 0
  else:  
     return 1 

In [9]:
# for i in range(15):
#   print(labels_seed[i][3][0])

In [10]:
features_sub  = []
labels_sub = []
n_window = 14
data_samples_sub = []
label_samples_sub = []
n_samples = []


for sub in range(15):
      print(f"processing {sub}")
      
      label = labels_seed[sub]
      data = data_seed[sub]
      #print(data[0][0, 0])
      data_samples = []
      label_samples = []
      n_data_samples = 0
      s = 0
      indexes = []
      for i in range(45):
        n = len(label[i])
        print(f"record{i}-{n}-{int(label[i][0])} ", end='')
        for sample, time in enumerate(range(0, n - n_window, n_window)):
          if (label[i][0] == 3):# or (label[i][0] == 0) or (label[i][0] == 1):
            continue
          data_sample = data[i][time : time + n_window]

          data_sample = (data_sample - data_sample.mean(axis = 1,  keepdims = True))#/features_all.std(axis = 1,  keepdims = True) + 0.5
          data_sample = (data_sample)/data_sample.std(axis = 1,  keepdims = True)
          data_sample = data_sample + 0.5
          data_sample = data_sample[np.newaxis, :, :]
          #print(data_sample.shape)#.transpose(np.newaxis, n_window, -1)
          data_samples.append(data_sample)
          n_data_samples += 1
          label_samples.append(get_label(label[i][0]))
      print()    
      print(label_samples)  
      print(n_data_samples)
      print("*********************************************")
      n_samples.append(n_data_samples)    
      data_samples_sub.append(data_samples)
      label_samples_sub.append(label_samples)


      
      # features_all = (features_all - features_all.mean(axis = 1,  keepdims = True))#/features_all.std(axis = 1,  keepdims = True) + 0.5
      # features_all = (features_all)/features_all.std(axis = 1,  keepdims = True)
      # features_all = features_all + 0.5
      # features_sub.append(features_all)
      # labels_sub.append(labels_all)

processing 0
record0-18-4 record1-24-1 record2-59-3 record3-46-2 record4-36-0 record5-64-4 record6-74-1 record7-17-3 record8-66-2 record9-35-0 record10-43-4 record11-43-1 record12-58-3 record13-60-2 record14-38-0 record15-59-2 record16-47-1 record17-16-3 record18-31-0 record19-32-4 record20-14-4 record21-60-0 record22-57-3 record23-30-2 record24-24-1 record25-46-3 record26-29-4 record27-23-1 record28-54-2 record29-19-0 record30-72-2 record31-16-1 record32-41-3 record33-22-0 record34-13-4 record35-59-4 record36-21-0 record37-18-3 record38-57-2 record39-71-1 record40-55-3 record41-29-4 record42-51-1 record43-32-2 record44-44-0 
[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
90
*********************************************
processing 1
record0-18-4 record1-24-1 record2

In [11]:
print(n_samples)

[90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90]


In [12]:
import glob
import pickle
from collections import Counter


class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.float32),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


transforms = [ToTensor()]   

class EmotionDataset(Dataset):
    def __init__ (self, out, n_windows, type):
        self.out = out
        self.type = type

    def __len__(self):
        if self.type == 'train':
            result = np.sum(n_samples)
            #print(result)
            result -= n_samples[self.out]

        else:
            result = n_samples[self.out]  
        return result
   
    def get_index(self, item):
        s = 0
        if self.type == 'train':
          for sub in range(15):
              if sub == self.out:
                  continue  
              if (item >= s) and (item < s + n_samples[sub]):
                 return sub, item - s
              s += n_samples[sub]
        else:
           return(self.out, item) 


    def __getitem__(self, item):
      sample = {}
      sub, index = self.get_index(item)
      
      sample['data'] = data_samples_sub[sub][index]
      sample['labels'] = label_samples_sub[sub][index]       
      #print(sub, index, sample['labels'] )

      if transforms is not None:
           for t in transforms:
                sample = t(sample)
       #print(sample)         
      return sample                                      
      

In [13]:
# все кроме 5
train_dataset = EmotionDataset(5, n_window, 'train')
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=True, drop_last=True)

# 5 человек
val_dataset = EmotionDataset(5, n_window, 'val')
val_dataloader = DataLoader(val_dataset, batch_size=4, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
 
 


In [14]:
for i, batch in enumerate(val_dataloader):
        continue

In [15]:
s = 0
for i, batch in enumerate(train_dataloader):#, t
        s += 1
        continue
print(f"**********{s}***********")        


**********157***********


The above emotion labels should be the same as labels in file "emotion_label_and_stimuli_order.xlsx"

In [16]:
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import pandas as pd


In [17]:
class EmotionNet(torch.nn.Module): 
   def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 32, kernel_size = 3)#, padding='same')
      self.relu1 = torch.nn.ReLU()
      self.pool1 = torch.nn.MaxPool2d(kernel_size = 2, stride=2)
      self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride=2)#, padding='same')
      self.relu2 = torch.nn.ReLU()
      self.pool2 = torch.nn.MaxPool2d(kernel_size = 2, stride=2)
      
      self.flat = torch.nn.Flatten(1, 3)
      #self.fc1 = torch.nn.Linear(512, 1024)
      self.fc1 = torch.nn.Linear(2432, 1024)
      self.relu3 = torch.nn.ReLU()
      self.fc2 = torch.nn.Linear(1024, 1)
      self.sig = nn.Sigmoid()
      
   def forward(self, input):
      output = self.conv1(input)
      #print(output.shape)
      output = self.relu1(output)
      output = self.pool1(output)
      #print(output.shape)
      output = self.conv2(output)
      #print(output.shape)
      output = self.relu2(output)
      output = self.pool2(output)
      #print(output.shape)
      #print(output.shape)
      output = self.flat(output)
      output = self.fc1(output)
      output = self.relu3(output)
      output = self.fc2(output)
      output = self.sig(output)
      #print(output.shape)
      
      return output.squeeze(1)



In [18]:
def train(model, loader, criterion, optimizer, device, type_emotion, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    #for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):
    for s, batch in enumerate(loader):#, t
    
            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels']

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            #print(levels_pred.shape)
            #print(trg.shape)
            loss = criterion(levels_pred, trg) 

            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs

In [19]:
def evaluate(model, loader, criterion, device, type_emotion):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        #for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
        for s, batch in enumerate(loader):#, t
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels']



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            
            loss = criterion(levels_pred, trg) 

            epoch_loss += loss.item() 
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report, f1_score

In [20]:
def calculate_predictions(model, loader, type_emotion, show):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        #for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
        for i, batch in enumerate(loader):#, t
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels']
           

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            
            #trg_pred = levels_pred.argmax(1)
            trg_pred = levels_pred > 0.5
            
            real.extend(trg)
            pred.extend(trg_pred) 

        if show:    
          print(accuracy_score(real, pred)) 
          print(confusion_matrix(real, pred))  
          print(classification_report(real, pred))   
        #print(real)
        #print(pred)  
    f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
    return (accuracy_score(real, pred)) , f1        
    

In [21]:
def get_model():
  model = EmotionNet().to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=3)
  (fc1): Linear(in_features=2432, out_features=1024, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=1024, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [22]:
def train_loop(type_emotion, description , nepochs = 5):
        args.epochs = nepochs
        #criterion =  fnn.mse_loss
        train_loss_min = 10000
        f1_min = -10000
        #batch = next(iter(train_dataloader))
        for epoch in range(args.epochs):
            #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
            
            train_loss = train(model, train_dataloader, criterion, optimizer ,device, type_emotion)
            #if epoch % 500 == 0:
            #print(f"train_loss = {train_loss}")

            if (train_loss < train_loss_min):
                train_loss_min      = train_loss
                torch.save({
                                'model_state_dict': model.state_dict(),
                                'optimizer_state_dict': optimizer.state_dict(),
                              },
                              os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", "train.tgz")
                    )  

            #val_loss = evaluate(model, val_dataloader, criterion, device, type_emotion)
            # #break
            #print(val_loss)
            #break

            acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, False)
            #print(f1, acc)
            if (f1 > f1_min):
              f1_min      =  f1
              torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/seed_models", f"{description}.tgz"))

In [24]:
type_emotion = 0
from sklearn.model_selection import StratifiedKFold
import pandas as pd
n_window = 14
args.batch_size = 128

acc_all = []
f1_all = []

for sub in range(0, 15): 
    print(sub) 
    train_dataset = EmotionDataset(sub, n_window, 'train')
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=True, drop_last=True)
    val_dataset = EmotionDataset(sub, n_window, 'val')
    val_dataloader = DataLoader(val_dataset, batch_size=4, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = get_model()
    model.apply(initialize_weights)
    #weight = torch.FloatTensor([1/27, 1/6])
   # criterion = nn.CrossEntropyLoss(weight = weight, reduction = 'mean')#torch.nn.MSELoss()
    #criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
    description = f'seed_common_{sub}'
    train_loop(type_emotion, description , 200)


    model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/seed_models/",  f"{description}.tgz"))
    #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
    model.load_state_dict(model_state['model_state_dict'])
    
    acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
    print(f"f1 = {f1} acc = {acc}")
    acc_all.append(acc)
    f1_all.append(f1)
    print(acc, f1)
    pd.DataFrame(f1_all).to_csv("f1_data_seed_result.csv")
    pd.DataFrame(acc_all).to_csv("acc_data_seed_result.csv")

0
0.8555555555555555
[[68  4]
 [ 9  9]]
              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91        72
         1.0       0.69      0.50      0.58        18

    accuracy                           0.86        90
   macro avg       0.79      0.72      0.75        90
weighted avg       0.84      0.86      0.85        90

f1 = 0.7466984195713358 acc = 0.8555555555555555
0.8555555555555555 0.7466984195713358
1
0.9111111111111111
[[71  1]
 [ 7 11]]
              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95        72
         1.0       0.92      0.61      0.73        18

    accuracy                           0.91        90
   macro avg       0.91      0.80      0.84        90
weighted avg       0.91      0.91      0.90        90

f1 = 0.8400000000000001 acc = 0.9111111111111111
0.9111111111111111 0.8400000000000001
2
0.9888888888888889
[[72  0]
 [ 1 17]]
              precision    recall  f1-score   suppor

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8111111111111111
[[72  0]
 [17  1]]
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89        72
         1.0       1.00      0.06      0.11        18

    accuracy                           0.81        90
   macro avg       0.90      0.53      0.50        90
weighted avg       0.85      0.81      0.74        90

f1 = 0.4998365478914678 acc = 0.8111111111111111
0.8111111111111111 0.4998365478914678
6
0.8
[[72  0]
 [18  0]]
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89        72
         1.0       0.00      0.00      0.00        18

    accuracy                           0.80        90
   macro avg       0.40      0.50      0.44        90
weighted avg       0.64      0.80      0.71        90

f1 = 0.4444444444444445 acc = 0.8
0.8 0.4444444444444445
7


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.0
[[72  0]
 [ 0 18]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        72
         1.0       1.00      1.00      1.00        18

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90

f1 = 1.0 acc = 1.0
1.0 1.0
8
0.9444444444444444
[[68  4]
 [ 1 17]]
              precision    recall  f1-score   support

         0.0       0.99      0.94      0.96        72
         1.0       0.81      0.94      0.87        18

    accuracy                           0.94        90
   macro avg       0.90      0.94      0.92        90
weighted avg       0.95      0.94      0.95        90

f1 = 0.9181669394435352 acc = 0.9444444444444444
0.9444444444444444 0.9181669394435352
9
0.8222222222222222
[[71  1]
 [15  3]]
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.90        72
         1.0       0